SQL 활용 데이터 프레임 생성

pymysql 과  sqlalchemy  패키지 미리 설치

In [ ]:
# ! pip install cryptography

In [ ]:
# 패키지 설치
import pymysql
from sqlalchemy import create_engine
from pandas import DataFrame
from pandas import read_sql, read_sql_table

#01. pymysql  사용

데이터베이스 접속\
django의 settings의 DATABASES 참고\
(docker에서 utf-8 설정 안해서 한글은 적용 안됨)

In [ ]:
dbcon = pymysql.connect(
    host="127.0.0.1",   #서버주소
    port = 3406,    #포트번호
    user='root',    #계정이름
    password='1234',    #비밀번호
    db='board', #DB이름
    charset='utf8'
)

 데이터 조회

In [ ]:
# 데이터 조회를 위한 커서 객체 생성
cursor = dbcon.cursor()

In [ ]:
# 데이터 조회를 위한 SQL문 처리
sql = "SELECT * FROM board_main_post"
cursor.execute(sql)
result = cursor.fetchall()
result

딕셔너리 형태로 데이터 조회

In [ ]:
# 데이터 조회를 위한 커서 객체 생성
cursor = dbcon.cursor(pymysql.cursors.DictCursor)

In [ ]:
# 데이터 조회하기
sql = "SELECT * FROM board_main_post"
cursor.execute(sql)
result = cursor.fetchall()
result

In [ ]:
# 조회결과를 데이터프레임으로 변환
df = DataFrame(result)
df

In [ ]:
# 데이터 프레임에 대한 인덱스 설정
# inplace=True를 입력해 반영
df.set_index('id', inplace=True)
df

입력, 수정, 삭제\
cursor 객체를 재사용

데이터 입력

In [ ]:
sql = """INSERT INTO board_main_post (title, contents, created_at, updated_at)
        VALUES ('pandas test', '이것은 테스트', now(), now())"""
print(sql)

rows = cursor.execute(sql)
print("%d 개의 행이 저장" %rows)
print("생성된 primary Key : %d" % cursor.lastrowid)

# 처리 결과를 반영하기 위한 commit
# insert, update, delete는 commit을 해야 반영
dbcon.commit()

# 되돌리기는 rollback() - 이미 commit한 내역은 rollback 불가

데이터 수정

In [ ]:
sql = """UPDATE board_main_post
        SET title = '제목을 수정',
        contents = '이거는 수정된 내용',
        updated_at=now()
        WHERE id=13"""
print(sql)

rows = cursor.execute(sql)
print("%d개의 행 갱신"%rows)

# 실제 반영을 위한 commit
dbcon.commit()

데이터 삭제

In [ ]:
sql = "DELETE FROM board_main_post WHERE id > 10"
print(sql)

rows = cursor.execute(sql)
print("%d개의 행이 삭제"%rows)

dbcon.commit()

데이터베이스 접속 해제

In [ ]:
cursor.close()
dbcon.close()
# close가 없으면 데이터가 계속 쌓여 서버가 다운됨

#02. SQLAlchemy 사용

DB 접속

In [ ]:
# 데이터베이스 접속 패키지 설치
pymysql.install_as_MySQLdb()
import MySQLdb

접속 문자열 생성

###### mysql+mysqldb://계정이름:비밀번호@:포트번호/데이터베이스이름?charset=인코딩
###### mysql+mysqldb://USER:PASSWORD@:PORT/NAME?charset=인코딩

In [ ]:
conStr = "mysql+mysqldb://root:1234@:3406/board?charset=utf8"
conStr

데이터베이스 접속

In [ ]:
engine = create_engine(conStr)
conn = engine.connect()

모든 데이터 조회

In [81]:
# SQL문 사용
df = read_sql("SELECT * FROM board_main_post", con=conn)
df

,id,title,contents,created_at,updated_at,author_id
0,1,1234,1234,2023-06-14 12:01:14.570250,2023-06-14 12:24:37.870945,None
1,2,poiu,poiu,2023-06-14 12:03:55.836730,2023-06-14 12:24:51.873327,None
2,4,qwe,qwe,2023-06-15 00:01:39.671555,2023-06-15 00:01:39.671583,None
3,5,my posta,first posta,2023-06-15 02:35:43.754412,2023-06-15 02:35:43.754412,None
4,8,asdf,asdf,2023-06-15 07:44:58.040802,2023-06-15 07:44:58.040802,None
5,9,1q2w3e4r5t,asdf,2023-06-15 07:45:48.736193,2023-06-15 08:29:03.941259,None
6,10,vvvv,vvvv\r\nsadfsadf,2023-06-15 07:46:00.733124,2023-06-15 08:03:44.607259,None


인덱스를 지정한 조회

In [82]:
df = read_sql_table('board_main_post', index_col='id', con=conn)
df

,title,contents,created_at,updated_at,author_id
id,,,,,
1,1234,1234,2023-06-14 12:01:14.570250,2023-06-14 12:24:37.870945,None
2,poiu,poiu,2023-06-14 12:03:55.836730,2023-06-14 12:24:51.873327,None
4,qwe,qwe,2023-06-15 00:01:39.671555,2023-06-15 00:01:39.671583,None
5,my posta,first posta,2023-06-15 02:35:43.754412,2023-06-15 02:35:43.754412,None
8,asdf,asdf,2023-06-15 07:44:58.040802,2023-06-15 07:44:58.040802,None
9,1q2w3e4r5t,asdf,2023-06-15 07:45:48.736193,2023-06-15 08:29:03.941259,None
10,vvvv,vvvv\r\nsadfsadf,2023-06-15 07:46:00.733124,2023-06-15 08:03:44.607259,None


특정 컬럼만 가져오기

In [87]:
df = read_sql_table("board_main_post", index_col='id', columns=['title', 'contents'], con=conn)
df

,title,contents
id,,
1,1234,1234
2,poiu,poiu
4,qwe,qwe
5,my posta,first posta
8,asdf,asdf
9,1q2w3e4r5t,asdf
10,vvvv,vvvv\r\nsadfsadf


데이터 내보내기

In [88]:
# f_exists='append' 옵션이 있으면, 
# 기존 테이블에 데이터를 추가로 삽입
df.to_sql(name="new_table", con=conn, if_exists="append", index=False)
conn.commit()

데이터베이스 접속 해제

In [91]:
conn.close()